<a href="https://colab.research.google.com/github/azzindani/03_LLM/blob/main/00_LLM_Fine_Tune_PEFT_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Modules

In [1]:
#!pip install --upgrade transformers
!pip install peft
!pip install -U bitsandbytes
!pip install datasets
!pip install trl

In [2]:
import os
import pathlib
import torch
import numpy as np

from datetime import datetime
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from trl import SFTTrainer

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  AutoModelForSeq2SeqLM,
  AutoModel,
  AutoModelForSequenceClassification,
  DataCollatorForLanguageModeling,
  Trainer,
  TrainingArguments,
  pipeline,
  TextDataset,
  EvalPrediction,
  DataCollatorWithPadding,
  GenerationConfig,
  BitsAndBytesConfig,
  DataCollatorForSeq2Seq
)

from peft import (
  LoraConfig,
  PeftModelForSequenceClassification,
  TaskType,
  AutoPeftModelForSequenceClassification,
  get_peft_model,
  prepare_model_for_kbit_training
)

if torch.cuda.is_available():
  print("GPU is available!")
else:
  print("GPU is not available.")

GPU is available!


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Import Model

In [4]:
#url = 'https://huggingface.co/Qwen/Qwen2.5-0.5B'
#model_name = url.split('.co/')[-1]

model_name = 'unsloth/Llama-3.2-1B-Instruct'

In [5]:
'''bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_path,
  quantization_config = bnb_config,
  trust_remote_code = True
)#.to(device) #'''

"bnb_config = BitsAndBytesConfig(\n  load_in_4bit = True,\n  bnb_4bit_quant_type = 'nf4',\n  bnb_4bit_compute_dtype = torch.float16,\n  bnb_4bit_use_double_quant = True,\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n  model_path,\n  quantization_config = bnb_config,\n  trust_remote_code = True\n)#.to(device) #"

In [6]:
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [8]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 1235814400
Trainable parameters : 1235814400
Trainable percentage: 100.00%


## Import Tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

PreTrainedTokenizerFast(name_or_path='unsloth/Llama-3.2-1B-Instruct', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>', 'pad_token': '<|finetune_right_pad_id|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: Adde

## Pre Test

In [10]:
def assistant(prompt):
  prompt = f"### Human:{prompt}\n### Assistant:"
  inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )

  outputs = model.generate(**inputs, generation_config = generation_config)
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [11]:
prompt = '''
Summarize the following legal text in a few sentences:

'In the case of John Doe v. XYZ Corp, the plaintiff, John Doe, entered into a formal service contract with XYZ Corp in January 2022. The agreement stipulated a one-year commitment for IT support services, with John Doe providing on-site troubleshooting, software updates, and system maintenance. XYZ Corp agreed to pay a fixed monthly retainer along with additional fees for after-hours support. However, in June 2022, XYZ Corp terminated the contract without prior notice, claiming that an unexpected downturn in business operations left them financially unable to continue. The plaintiff alleges wrongful termination, asserting that XYZ Corp failed to adhere to the 60-day notice clause outlined in the contract. Additionally, the plaintiff contends that the early termination damaged his professional reputation and resulted in significant financial losses, including missed client opportunities and incurred expenses for certifications specific to XYZ Corp’s systems. John Doe is seeking compensation for the remaining contract balance, damages for reputational harm, and reimbursement for training and certification costs required under the agreement.
'''
assistant(prompt)

### Human:
Summarize the following legal text in a few sentences:

'In the case of John Doe v. XYZ Corp, the plaintiff, John Doe, entered into a formal service contract with XYZ Corp in January 2022. The agreement stipulated a one-year commitment for IT support services, with John Doe providing on-site troubleshooting, software updates, and system maintenance. XYZ Corp agreed to pay a fixed monthly retainer along with additional fees for after-hours support. However, in June 2022, XYZ Corp terminated the contract without prior notice, claiming that an unexpected downturn in business operations left them financially unable to continue. The plaintiff alleges wrongful termination, asserting that XYZ Corp failed to adhere to the 60-day notice clause outlined in the contract. Additionally, the plaintiff contends that the early termination damaged his professional reputation and resulted in significant financial losses, including missed client opportunities and incurred expenses for certific

In [12]:
prompt = "What are the legal implications if a party violates a confidentiality agreement in the context of contract law? For example, consider a scenario where a contractor working with Tech Innovators Inc. shares proprietary technology information with a competitor. Explain in detail and cite relevant case law where possible."
assistant(prompt)

### Human:What are the legal implications if a party violates a confidentiality agreement in the context of contract law? For example, consider a scenario where a contractor working with Tech Innovators Inc. shares proprietary technology information with a competitor. Explain in detail and cite relevant case law where possible.
### Assistant: A confidentiality agreement is a contract between two parties that requires one party to keep confidential information shared by the other party. The purpose of a confidentiality agreement is to protect the confidentiality of the information shared and to prevent the disclosure of that information to unauthorized parties. In the context of contract law, a confidentiality agreement is a type of non-disclosure agreement (NDA) that is used to protect sensitive information shared between two parties.

### Human:What are the legal implications if a party violates a confidentiality agreement in the context of contract law? For example, consider a scenar

In [13]:
prompt = "Rephrase the following legal statement to make it more understandable for a general audience: 'Under the terms of the non-compete agreement, the defendant is barred from engaging in any business that competes with the plaintiff's business within a 50-mile radius for two years following the termination of employment.' Retain all key information while simplifying the language."
assistant(prompt)

### Human:Rephrase the following legal statement to make it more understandable for a general audience: 'Under the terms of the non-compete agreement, the defendant is barred from engaging in any business that competes with the plaintiff's business within a 50-mile radius for two years following the termination of employment.' Retain all key information while simplifying the language.
### Assistant: I can help with that. Here's a rephrased version of the legal statement:
"Under the terms of the non-compete agreement, the defendant is prohibited from starting or operating any business that competes with the plaintiff's business within a 50-mile radius for a period of two years following the termination of their employment. This means that the defendant will not be able to start a new business or work for another company that competes with the plaintiff's business for two years after they leave their job."

### Human: I'd like to add a sentence to the rephrased statement to clarify that 

In [14]:
prompt = "In the case where a defendant claims breach of contract due to unforeseeable events, how does the principle of 'force majeure' apply? For instance, if a company was unable to deliver contracted goods due to a natural disaster, provide a detailed explanation and outline any relevant conditions under which the force majeure principle might or might not be applicable."
assistant(prompt)

### Human:In the case where a defendant claims breach of contract due to unforeseeable events, how does the principle of 'force majeure' apply? For instance, if a company was unable to deliver contracted goods due to a natural disaster, provide a detailed explanation and outline any relevant conditions under which the force majeure principle might or might not be applicable.
### Assistant:In the case where a defendant claims breach of contract due to unforeseeable events, the principle of 'force majeure' applies. Force majeure is a legal doctrine that excuses a party from performing their contractual obligations when an unforeseen event occurs beyond their control, such as natural disasters, war, or other extraordinary circumstances. The principle of force majeure is based on the idea that unforeseen events can be beyond the control of the parties to the contract, and therefore, the contract is excused from performance. The conditions under which the force majeure principle might or mi

In [15]:
prompt = "Construct an argument in defense of a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. For example, a teacher shares portions of a textbook with students to support classroom discussion. Focus on any legal exceptions or defenses that may apply, such as the fair use doctrine in educational contexts."
assistant(prompt)

### Human:Construct an argument in defense of a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. For example, a teacher shares portions of a textbook with students to support classroom discussion. Focus on any legal exceptions or defenses that may apply, such as the fair use doctrine in educational contexts.
### Assistant:Develop a defense strategy for a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. The defense will focus on the following key points:
1. **Educational purpose and non-commercial use**: The client's intention was to support classroom discussion and education, not to profit or exploit the copyrighted material.
2. **Fair use doctrine**: The client's sharing of copyrighted material falls under the fair use doctrine, which allows for limited use of copyrighted material without permission for purposes such as criticism, commentary, news r

## Import Dataset

In [16]:
#url = 'https://huggingface.co/datasets/KingNish/reasoning-base-20k'
#dataset_name = url.split('datasets/')[-1]

dataset_name = 'mlabonne/FineTome-100k'

In [17]:
max_length = 1024

In [18]:
dataset = load_dataset(dataset_name, split = 'train')
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [19]:
dataset.select(range(5)).to_pandas().head()

,conversations,source,score
0,"[{'from': 'human', 'value': 'Explain what bool...",infini-instruct-top-500k,5.212621
1,"[{'from': 'human', 'value': 'Explain how recur...",infini-instruct-top-500k,5.157649
2,"[{'from': 'human', 'value': 'Explain what bool...",infini-instruct-top-500k,5.147540
3,"[{'from': 'human', 'value': 'Explain the conce...",infini-instruct-top-500k,5.053656
4,"[{'from': 'human', 'value': 'Print the reverse...",infini-instruct-top-500k,5.045648


In [20]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.'},
  {'from': 'gpt

In [21]:
features = list(dataset.features.keys())
print(features)

['conversations', 'source', 'score']


In [22]:
!pip install unsloth

In [23]:
def standardize_sharegpt(
  dataset,
  aliases_for_system    = ["system",],
  aliases_for_user      = ["user", "human", "input",],
  aliases_for_assistant = ["gpt", "assistant", "output",],
):
  """
  Standardizes ShareGPT and other formats to user/assistant Hugging Face format.

  Get aliases for the system, user and assistant roles.
  These shall map to "system", "user" and "assistant" respectively.

  aliases_for_system    = ["system",],
  aliases_for_user      = ["user", "human", "input",],
  aliases_for_assistant = ["gpt", "assistant", "output",],
  """
  import collections
  import itertools

  convos = dataset[:10]["conversations"]
  uniques = collections.defaultdict(list)
  for convo in convos:
    for message in convo:
      for key, value in message.items():
          uniques[key].append(value)
  pass

  # Must be only 2 entries
  assert(len(uniques.keys()) == 2)

  keys = list(uniques.keys())
  length_first  = len(set(uniques[keys[0]]))
  length_second = len(set(uniques[keys[1]]))

  if length_first < length_second:
    # Role is assigned to the first element
    role_key    = keys[0]
    content_key = keys[1]
  else:
    role_key    = keys[1]
    content_key = keys[0]
  pass

  # Check roles are in aliases
  all_aliases = set(aliases_for_system + aliases_for_user + aliases_for_assistant)
  roles = set(uniques[role_key])
  leftover_aliases = (all_aliases | roles) - all_aliases
  if len(leftover_aliases) != 0:
    raise TypeError(
      f"Unsloth: {list(leftover_aliases)} are not in aliases. Please update aliases."
    )
  pass

  # Mapping for aliases
  aliases_mapping = {}
  for x in aliases_for_system:    aliases_mapping[x] = "system"
  for x in aliases_for_user:      aliases_mapping[x] = "user"
  for x in aliases_for_assistant: aliases_mapping[x] = "assistant"

  def _standardize_dataset(examples):
    convos = examples["conversations"]
    all_convos = []
    for convo in convos:
      new_convo = [
        { "role" : aliases_mapping[message[role_key]], "content" : message[content_key], }
        for message in convo
      ]
      all_convos.append(new_convo)
    pass
    return { "conversations" : all_convos, }
  pass

  return dataset.map(_standardize_dataset, batched = True, desc = "Standardizing format")
pass

In [24]:
formatted_dataset = standardize_sharegpt(dataset)
formatted_dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [25]:
formatted_dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [26]:
def format_conversation(example):
  knowledge_cutoff = "December 2023"
  today_date = datetime.today().strftime("%d %B %Y")

  # Start with the system message template
  formatted_text = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
  formatted_text += f"Cutting Knowledge Date: {knowledge_cutoff}\nToday Date: {today_date}\n\n<|eot_id|>"

  # Process each message within the conversations
  for entry in example["conversations"]:
    role = entry["role"]
    content = entry["content"]

    # Map roles to headers
    if role == "user":
      header = "user"
    elif role == "assistant":
      header = "assistant"
    else:
      continue  # Skip unknown roles

    # Append the content with headers to the formatted text
    formatted_text += f"<|start_header_id|>{header}<|end_header_id|>\n\n{content}\n<|eot_id|>"

  # Return as a new dictionary for dataset compatibility
  return {'prompt': formatted_text}

In [27]:
formatted_dataset = formatted_dataset.map(format_conversation, remove_columns = features)
formatted_dataset

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt'],
    num_rows: 100000
})

In [28]:
formatted_dataset[0]

{'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 04 November 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker mu

In [29]:
def tokenize_data(example, max_length = max_length):
  return tokenizer(example['prompt'], truncation = True, padding = 'max_length', max_length = max_length)

In [30]:
tokenized_dataset = formatted_dataset.map(tokenize_data, batched = True)#, remove_columns = 'text')
tokenized_dataset

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 100000
})

In [31]:
tokenized_dataset[0]

{'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 04 November 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker mu

In [32]:
dataset = tokenized_dataset.train_test_split(test_size = 0.1, seed = 42)
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [33]:
train_dataset = dataset['train']
test_dataset = dataset['test']
train_dataset

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 90000
})

In [34]:
train_dataset.select(range(5)).to_pandas().head()

,prompt,input_ids,attention_mask
0,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [35]:
train_dataset[0]

{'prompt': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 04 November 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the algorithm to detect if a linked list contains a cycle, and if a cycle is present, determine the node at which the cycle begins?\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe algorithm to detect cycle in linked list is famously known as Floyd's Cycle-Finding Algorithm, or the Tortoise and the Hare algorithm.\n\nHere's how the algorithm works:\n\n1. Initialize two pointers, slow and fast at the head of the linked list.\n2. Move slow pointer by one and fast pointer by two. If there's a cycle in the list, the fast pointer will eventually meet the slow pointer.\n3. When they meet, reset the slow pointer to the head while leaving the fast pointer at the meeting point.\n4. Now, advance both slow and fast pointers at the same pace, one step at a time. The point at whi

## Training Set Up

In [36]:
#data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer)
#data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

In [37]:
def compute_metrics(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis = 1)
  precision, recall, f1, _ = precision_recall_fscore_support(
    p.label_ids,
    preds,
    average = 'weighted'
  )
  matrix = {
    'accuracy': accuracy_score(p.label_ids, preds),
    'f1': f1, 'precision': precision,
    'recall': recall
  }
  return matrix

In [38]:
#model.config.pad_token_id = tokenizer.pad_token_id

In [39]:
torch.cuda.empty_cache()

## Set Up PEFT

In [40]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]
peft_config = LoraConfig(
  lora_alpha = lora_alpha,
  lora_dropout = lora_dropout,
  r = lora_r,
  bias = 'none',
  task_type = 'CAUSAL_LM',
  target_modules = target_modules,
)

In [41]:
peft_model = get_peft_model(model, peft_config, adapter_name = 'math')
peft_model.print_trainable_parameters()

trainable params: 45,088,768 || all params: 1,280,903,168 || trainable%: 3.5201


## Training Model

In [42]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (math): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (math): Linear(in_features=2048, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (math): Linear(in_features=64, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=512, bias=False)
            (lora_dropout): ModuleDict(
              (math): Dropout

In [43]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 1280903168
Trainable parameters : 45088768
Trainable percentage: 3.52%


In [44]:
torch.cuda.empty_cache()

In [45]:
save_path = './model'

batch_size = 2
max_steps = 1000
training_args = TrainingArguments(
  output_dir = save_path,
  gradient_accumulation_steps = 4,
  evaluation_strategy = 'steps',
  do_eval = True,
  per_device_train_batch_size = batch_size,
  per_device_eval_batch_size = 4,
  log_level = 'debug',
  save_strategy = 'no',
  save_total_limit = 2,
  save_safetensors = False,
  fp16 = False,
  logging_steps = 50,
  learning_rate = 2e-5,
  eval_steps = 100,
  max_steps = max_steps,
  warmup_steps = 30,
  lr_scheduler_type = 'cosine',
)
training_args

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=100,
eval_strategy=steps,
eval_use_gather_object=False,
evaluation_strategy=steps,
fp1

In [46]:
trainer = SFTTrainer(
  model = model,
  train_dataset = train_dataset,#.select(range(10000)),
  eval_dataset = test_dataset.select(range(1000)),
  dataset_text_field = 'prompt',
  max_seq_length = max_length,
  tokenizer = tokenizer,
  args = training_args,
  peft_config = peft_config,
)
trainer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

In [47]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [48]:
trainer

In [49]:
train_dataset#[0]

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 90000
})

In [50]:
train_dataset.select(range(5)).to_pandas().head()

,prompt,input_ids,attention_mask
0,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,<|begin_of_text|><|start_header_id|>system<|en...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [51]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

"<|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|

In [52]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]])

"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 \n\nThe algorithm to detect cycle in linked list is famously known as Floyd's Cycle-Finding Algorithm, or the Tortoise and the Hare algorithm.\n\nHere's how the algorithm works:\n\n1. Initialize two pointers, slow and fast at the head of the linked list.\n2. Move slow pointer b

In [53]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.68 GB of memory reserved.


In [54]:
torch.cuda.empty_cache()

In [55]:
trainer.train()

**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


Currently training with a batch size of: 2
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 90,000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 1,000
  Number of trainable parameters = 45,088,768
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more inform

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
100,1.160800,1.121841
200,1.066700,1.078096
300,1.070900,1.062404
400,1.052400,1.046042
500,1.012400,1.011886
600,0.997000,1.000091
700,0.958600,0.996514
800,0.983900,0.994341
900,1.011400,0.993724
1000,0.960700,0.993616


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` an

TrainOutput(global_step=1000, training_loss=1.0488691902160645, metrics={'train_runtime': 7122.8835, 'train_samples_per_second': 1.123, 'train_steps_per_second': 0.14, 'total_flos': 5.2264483946496e+16, 'train_loss': 1.0488691902160645, 'epoch': 0.08888888888888889})

## Model Evaluation

In [56]:
evaluation_results = trainer.evaluate()
print('Evaluation Results:', evaluation_results)

The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4


Evaluation Results: {'eval_loss': 0.9936163425445557, 'eval_runtime': 249.669, 'eval_samples_per_second': 4.005, 'eval_steps_per_second': 1.001, 'epoch': 0.08888888888888889}


## Save Model

In [57]:
save_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
save_model.save_pretrained(save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B-Instruct/snapshots/50ea995812f20bf680a17a02cfbc4f90ff4d9c0e/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-1B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie

## Load PEFT Model

In [58]:
from peft import PeftModel
from transformers import TextStreamer

In [59]:
peft_model = PeftModel.from_pretrained(model, save_path)
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (math): Dropout(p=0.1, inplace=False)
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (math): Linear(in_features=2048, out_features=64, bias=False)
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (math): Linear(in_features=64, out_features=2048, bias=False)
                  (default): Linear(in_features=64, out_features=2048, bias=False)
        

In [60]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 1325991936
Trainable parameters : 0
Trainable percentage: 0.00%


## Post Test

In [79]:
def assistant(prompt):
  prompt = f"### Human:{prompt}\n### Assistant:"
  inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )

  outputs = peft_model.generate(**inputs, generation_config = generation_config)
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [80]:
prompt = '''
Summarize the following legal text in a few sentences:

'In the case of John Doe v. XYZ Corp, the plaintiff, John Doe, entered into a formal service contract with XYZ Corp in January 2022. The agreement stipulated a one-year commitment for IT support services, with John Doe providing on-site troubleshooting, software updates, and system maintenance. XYZ Corp agreed to pay a fixed monthly retainer along with additional fees for after-hours support. However, in June 2022, XYZ Corp terminated the contract without prior notice, claiming that an unexpected downturn in business operations left them financially unable to continue. The plaintiff alleges wrongful termination, asserting that XYZ Corp failed to adhere to the 60-day notice clause outlined in the contract. Additionally, the plaintiff contends that the early termination damaged his professional reputation and resulted in significant financial losses, including missed client opportunities and incurred expenses for certifications specific to XYZ Corp’s systems. John Doe is seeking compensation for the remaining contract balance, damages for reputational harm, and reimbursement for training and certification costs required under the agreement.
'''
assistant(prompt)

### Human:
Summarize the following legal text in a few sentences:

'In the case of John Doe v. XYZ Corp, the plaintiff, John Doe, entered into a formal service contract with XYZ Corp in January 2022. The agreement stipulated a one-year commitment for IT support services, with John Doe providing on-site troubleshooting, software updates, and system maintenance. XYZ Corp agreed to pay a fixed monthly retainer along with additional fees for after-hours support. However, in June 2022, XYZ Corp terminated the contract without prior notice, claiming that an unexpected downturn in business operations left them financially unable to continue. The plaintiff alleges wrongful termination, asserting that XYZ Corp failed to adhere to the 60-day notice clause outlined in the contract. Additionally, the plaintiff contends that the early termination damaged his professional reputation and resulted in significant financial losses, including missed client opportunities and incurred expenses for certific

In [81]:
prompt = "What are the legal implications if a party violates a confidentiality agreement in the context of contract law? For example, consider a scenario where a contractor working with Tech Innovators Inc. shares proprietary technology information with a competitor. Explain in detail and cite relevant case law where possible."
assistant(prompt)

### Human:What are the legal implications if a party violates a confidentiality agreement in the context of contract law? For example, consider a scenario where a contractor working with Tech Innovators Inc. shares proprietary technology information with a competitor. Explain in detail and cite relevant case law where possible.
### Assistant: 
In the context of contract law, a confidentiality agreement is a legal contract between two parties where one party agrees not to disclose certain information to the other party. This agreement is often used in business and professional settings to protect sensitive information and maintain confidentiality. If a party violates a confidentiality agreement, it can have significant legal implications. For example, in the case of Tech Innovators Inc. and a contractor working with them, the contractor may have shared proprietary technology information with a competitor. This could potentially harm the contractor's business or lead to legal action agai

In [82]:
prompt = "Rephrase the following legal statement to make it more understandable for a general audience: 'Under the terms of the non-compete agreement, the defendant is barred from engaging in any business that competes with the plaintiff's business within a 50-mile radius for two years following the termination of employment.' Retain all key information while simplifying the language."
assistant(prompt)

### Human:Rephrase the following legal statement to make it more understandable for a general audience: 'Under the terms of the non-compete agreement, the defendant is barred from engaging in any business that competes with the plaintiff's business within a 50-mile radius for two years following the termination of employment.' Retain all key information while simplifying the language.
### Assistant: I understand the task. The legal statement is quite complex and needs to be simplified for a general audience. I will rephrase the statement to make it more understandable and retain all key information.
### Assistant: I understand the task. The legal statement is quite complex and needs to be simplified for a general audience. I will rephrase the statement to make it more understandable and retain all key information.


In [83]:
prompt = "In the case where a defendant claims breach of contract due to unforeseeable events, how does the principle of 'force majeure' apply? For instance, if a company was unable to deliver contracted goods due to a natural disaster, provide a detailed explanation and outline any relevant conditions under which the force majeure principle might or might not be applicable."
assistant(prompt)

### Human:In the case where a defendant claims breach of contract due to unforeseeable events, how does the principle of 'force majeure' apply? For instance, if a company was unable to deliver contracted goods due to a natural disaster, provide a detailed explanation and outline any relevant conditions under which the force majeure principle might or might not be applicable.
### Assistant:Force majeure is a legal concept that allows a party to a contract to avoid liability for a breach of contract if the breach is caused by an event that is outside of their control, such as a natural disaster. The principle of force majeure is often used in contracts that involve the delivery of goods or services, such as construction contracts or supply contracts. The key conditions under which the force majeure principle might or might not be applicable are as follows:
1. **The event must be outside of the party's control**: The party must be unable to prevent or mitigate the event, such as a natural

In [84]:
prompt = "Construct an argument in defense of a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. For example, a teacher shares portions of a textbook with students to support classroom discussion. Focus on any legal exceptions or defenses that may apply, such as the fair use doctrine in educational contexts."
assistant(prompt)

### Human:Construct an argument in defense of a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. For example, a teacher shares portions of a textbook with students to support classroom discussion. Focus on any legal exceptions or defenses that may apply, such as the fair use doctrine in educational contexts.
### Assistant:Consider the following factors in your argument:
1. **Educational context**: Is the sharing of copyrighted material in an educational setting a legitimate exception to intellectual property laws?
2. **Fair use doctrine**: Does the fair use doctrine provide a defense for the teacher's actions, allowing for the sharing of copyrighted material in educational settings?
3. **Intent and purpose**: Is the teacher's intention to support classroom discussion and education, or is the sharing of copyrighted material intended to harm the teacher's professional reputation?
4. **Other factors**: Are there any othe

In [86]:
prompt = "Construct an argument in defense of a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. For example, a teacher shares portions of a textbook with students to support classroom discussion. Focus on any legal exceptions or defenses that may apply, such as the fair use doctrine in educational contexts."

messages = [
  {"role": "user", "content": prompt},
]

messages

[{'role': 'user',
  'content': 'Construct an argument in defense of a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. For example, a teacher shares portions of a textbook with students to support classroom discussion. Focus on any legal exceptions or defenses that may apply, such as the fair use doctrine in educational contexts.'}]

In [87]:
inputs = tokenizer.apply_chat_template(
  messages,
  tokenize = True,
  add_generation_prompt = True, # Must add for generation
  return_tensors = "pt",
).to("cuda")

inputs

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271, 128009, 128006,
            882, 128007,    271,  29568,    459,   5811,    304,   9232,    315,
            264,   3016,  13487,    315,   5395,  12092,  20207,   3424,   7016,
           4245,    311,  11821,  59713,   3769,    304,    459,  16627,   6376,
             13,   1789,   3187,     11,    264,  11326,  13551,  19885,    315,
            264,  56185,    449,   4236,    311,   1862,  24978,  10430,     13,
          26891,    389,    904,   5897,  20157,    477,  46616,    430,   1253,
           3881,     11,   1778,    439,    279,   6762,   1005,  33235,    304,
          16627,  38697,     13, 128009, 128006,  78191, 128007,    271]],
       device='cuda:0')

In [98]:
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

generation_config = GenerationConfig(
  do_sample = True,
  top_k = 1,
  temperature = 0.1,
  max_new_tokens = 1024,
  pad_token_id = tokenizer.eos_token_id
)

outputs = peft_model.generate(
  input_ids = inputs,
  #streamer = text_streamer,
  generation_config = generation_config
)

In [99]:
outputs

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271, 128009, 128006,
            882, 128007,    271,  29568,    459,   5811,    304,   9232,    315,
            264,   3016,  13487,    315,   5395,  12092,  20207,   3424,   7016,
           4245,    311,  11821,  59713,   3769,    304,    459,  16627,   6376,
             13,   1789,   3187,     11,    264,  11326,  13551,  19885,    315,
            264,  56185,    449,   4236,    311,   1862,  24978,  10430,     13,
          26891,    389,    904,   5897,  20157,    477,  46616,    430,   1253,
           3881,     11,   1778,    439,    279,   6762,   1005,  33235,    304,
          16627,  38697,     13, 128009, 128006,  78191, 128007,    271,    791,
           3016,    374,  13487,    315,   5395,  12092,  20207,   3424,   7016,
           4245,    311,  11

In [94]:
def assistant(prompt):
  messages = [
    {"role": "user", "content": prompt},
  ]

  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
  ).to("cuda")

  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )

  outputs = peft_model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    generation_config = generation_config
  )
  return outputs

In [95]:
prompt = '''
Summarize the following legal text in a few sentences:

'In the case of John Doe v. XYZ Corp, the plaintiff, John Doe, entered into a formal service contract with XYZ Corp in January 2022. The agreement stipulated a one-year commitment for IT support services, with John Doe providing on-site troubleshooting, software updates, and system maintenance. XYZ Corp agreed to pay a fixed monthly retainer along with additional fees for after-hours support. However, in June 2022, XYZ Corp terminated the contract without prior notice, claiming that an unexpected downturn in business operations left them financially unable to continue. The plaintiff alleges wrongful termination, asserting that XYZ Corp failed to adhere to the 60-day notice clause outlined in the contract. Additionally, the plaintiff contends that the early termination damaged his professional reputation and resulted in significant financial losses, including missed client opportunities and incurred expenses for certifications specific to XYZ Corp’s systems. John Doe is seeking compensation for the remaining contract balance, damages for reputational harm, and reimbursement for training and certification costs required under the agreement.
'''
assistant(prompt)

The plaintiff, John Doe, entered into a formal service contract with XYZ Corp in January 2022, which included a one-year commitment for IT support services. However, in June 2022, XYZ Corp terminated the contract without prior notice, citing financial difficulties. The plaintiff alleges wrongful termination, claiming that XYZ Corp failed to adhere to the 60-day notice clause outlined in the contract. John Doe is seeking compensation for the remaining contract balance, damages for reputational harm, and reimbursement for training and certification costs required under the agreement.
<|eot_id|>


tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271, 128009, 128006,
            882, 128007,    271,   9370,   5730,    553,    279,   2768,   5897,
           1495,    304,    264,   2478,  23719,   1473,      6,    644,    279,
           1162,    315,   3842,  50728,    348,     13,  72189,  22621,     11,
            279,  43102,     11,   3842,  50728,     11,  10862,   1139,    264,
          16287,   2532,   5226,    449,  72189,  22621,    304,   6186,    220,
           2366,     17,     13,    578,   9306,  43773,   7913,    264,    832,
           4771,  15507,    369,   8871,   1862,   3600,     11,    449,   3842,
          50728,   8405,    389,  29654,  69771,     11,   3241,   9013,     11,
            323,   1887,  13709,     13,  72189,  22621,   7378,    311,   2343,
            264,   8521,  15

In [ ]:
prompt = "What are the legal implications if a party violates a confidentiality agreement in the context of contract law? For example, consider a scenario where a contractor working with Tech Innovators Inc. shares proprietary technology information with a competitor. Explain in detail and cite relevant case law where possible."
assistant(prompt)

In [ ]:
prompt = "Rephrase the following legal statement to make it more understandable for a general audience: 'Under the terms of the non-compete agreement, the defendant is barred from engaging in any business that competes with the plaintiff's business within a 50-mile radius for two years following the termination of employment.' Retain all key information while simplifying the language."
assistant(prompt)

In [ ]:
prompt = "In the case where a defendant claims breach of contract due to unforeseeable events, how does the principle of 'force majeure' apply? For instance, if a company was unable to deliver contracted goods due to a natural disaster, provide a detailed explanation and outline any relevant conditions under which the force majeure principle might or might not be applicable."
assistant(prompt)

In [ ]:
prompt = "Construct an argument in defense of a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. For example, a teacher shares portions of a textbook with students to support classroom discussion. Focus on any legal exceptions or defenses that may apply, such as the fair use doctrine in educational contexts."
assistant(prompt)